In [ ]:
import numpy as np
import pandas as pd


import os

import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.applications import Xception
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D,GlobalAveragePooling2D
from keras.optimizers import Adam,Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Model,Sequential
from tensorflow.keras.layers import Input

c:\Users\naelb\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
IMAGE_SIZE = 80
COLORS=3
BATCH_SIZE = 64
RANDOM_STATE=42

In [13]:
classification_types=['Pepper__bell___Bacterial_spot','Pepper__bell___healthy',
                      'Potato___Early_blight','Potato___Late_blight','Potato___healthy',
                      'Tomato_Bacterial_spot','Tomato_Early_blight','Tomato_Late_blight',
                      'Tomato_Leaf_Mold','Tomato_Septoria_leaf_spot',
                      'Tomato_Spider_mites_Two_spotted_spider_mite','Tomato__Target_Spot',
                      'Tomato__Tomato_YellowLeaf__Curl_Virus',
                      'Tomato__Tomato_mosaic_virus','Tomato_healthy']
data_dir = 'PlantVillage/'
train_dir = os.path.join(data_dir)
print(train_dir)

PlantVillage/


In [15]:
train_data=[]
for id, disease in enumerate(classification_types):
    dir = os.path.join(data_dir,disease)
    for file in os.listdir(dir):
        train_data.append([os.path.join(disease,file),id,disease,os.path.join(dir,file)])


train = pd.DataFrame(train_data,columns=['File','DiseaseID','Disease_Type','File_Path'])
train.head()

,File,DiseaseID,Disease_Type,File_Path
0,Pepper__bell___Bacterial_spot\0022d6b7-d47c-4e...,0,Pepper__bell___Bacterial_spot,PlantVillage/Pepper__bell___Bacterial_spot\002...
1,Pepper__bell___Bacterial_spot\006adb74-934f-44...,0,Pepper__bell___Bacterial_spot,PlantVillage/Pepper__bell___Bacterial_spot\006...
2,Pepper__bell___Bacterial_spot\00f2e69a-1e56-41...,0,Pepper__bell___Bacterial_spot,PlantVillage/Pepper__bell___Bacterial_spot\00f...
3,Pepper__bell___Bacterial_spot\01613cd0-d3cd-4e...,0,Pepper__bell___Bacterial_spot,PlantVillage/Pepper__bell___Bacterial_spot\016...
4,Pepper__bell___Bacterial_spot\0169b9ac-07b9-4b...,0,Pepper__bell___Bacterial_spot,PlantVillage/Pepper__bell___Bacterial_spot\016...
